
Name: Rachel Fanti Coelho Lima    
Date: 02/2022     
Subject: LM-18 73005B - Data Profiling - AY 2021-22    
Task: PLI - duplicated.


In [8]:
%matplotlib inline

In [9]:
# pip install PyQt5==5.9.2

In [10]:
import time
import pandas as pd
import networkx as nx
import matplotlib.pyplot as plt
import seaborn
import sys

#pip install pandas_profiling --user
#pip install PyQt5==5.9.2
from pandas_profiling import ProfileReport
from pandas_profiling.utils.cache import cache_file

### Input Data 

In [11]:
#df = pd.DataFrame([ [ 'C', '3', 'X', 722, 112 ],
#                    [ 'A', '1', 'X', 289, 553 ],
#                    [ 'A', '1', 'Y', 189, 583 ],
#                    [ 'B', '1', 'X', 289, 513 ],
#                    [ 'C', '1', 'X', 289, 553 ] ], columns = [a for a in 'abcde'])

In [12]:
df = pd.read_csv('./../00_data/Wines-2.csv', sep = ",", index_col=False)
df.columns=['a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o']

In [13]:
df.head()

,a,b,c,d,e,f,g,h,i,j,k,l,m,n,o
0,Taurasi,Taurasi,Taurasi,Agricola Boccella,Color rubino limpido; l’olfatto è subito inten...,100% Aglianico,"14,5%",D.O.C.G.,"Villaggio S. Eustachio, 16",Castelfranci,Avellino,"40.927652, 15.052481",http://www.boccellavini.it/,0827 72574,www.vinix.com
1,Casefatte,Campania,Bianco,Agricola Boccella,"Di colore giallo paglierino, profumi agrumati,...",100% Fiano,13%,I.G.T.,"Villaggio S. Eustachio, 16",Castelfranci,Avellino,"40.927652, 15.052481",http://www.boccellavini.it/,0827 72574,http://www.doctorwine.it/
2,Rasott,Irpinia Campi Taurasini,Rosso,Agricola Boccella,Color rubino limpido; l’olfatto è subito inten...,100% Aglianico,14%,D.O.C.,"Villaggio S. Eustachio, 16",Castelfranci,Avellino,"40.927652, 15.052481",http://www.boccellavini.it/,0827 72574,www.vinix.com
3,Sannio Falanghina,Falanghina del Sannio,Bianco,Agricola Irpina,Colore giallo paglierino con riflessi verdogno...,100% Falanghina,NaN,D.O.C.,"Via Limaturo, 48",San Michele di Pratola Serra,Avellino,"40.987214, 14.865230",http://www.agricolairpina.com/it/chi_siamo.php,0825 967038,NaN
4,Irpinia Aglianico,Irpinia,Rosso,Agricola Irpina,"Di colore rosso rubino, ha profumo intenso, sa...",85-90% Aglianico; 15-10% Merlot o Sangiovese,NaN,D.O.C.,"Via Limaturo, 48",San Michele di Pratola Serra,Avellino,"40.987214, 14.865230",http://www.agricolairpina.com/it/chi_siamo.php,0825 967038,NaN


In [14]:
schema = set(a for a in df.columns)
#schema

### Helper functions

In [15]:
def sort_single(s):
    return ''.join(sorted(list(s)))

def sort(F):
    return sorted(sort_single(s) for s in F)

In [16]:
def pli_single(a, D):
    pli = {}
    for row in range(D.shape[0]):
        key = D.at[row, a]
        if key in pli:
            pli[key].add(row)
        else:
            pli[key] = { row }
    return list(v for v in pli.values() if len(v) > 1)

In [17]:
def pli_intersect(p1, p2):
    pli, prob = {}, {}
    for c, s in enumerate(p1):
        for row in s:
            prob[row] = c
    for c, s in enumerate(p2):
        for row in s:
            if row in prob:
                key = (c, prob[row])
                if key in pli:
                    pli[key].add(row)
                else:
                    pli[key] = { row }
            
    return list(v for v in pli.values() if len(v) > 1)

### Duplicated records (PLI)

In [18]:
table = df

In [19]:
pli = {a: pli_single(a, table) for a in schema}
#pli

In [20]:
k=2
while k<= len (df.columns):
    columns_pli = [i for i in df.columns[0:k]]
    pli [sort_single(columns_pli)] = pli_intersect(pli[sort_single(columns_pli[:-1])], pli[columns_pli[-1]])
    #= pli_intersect(pli['ab'], pli['d'])
    k = k+1
print ('pli', sort_single(df.columns))
print (pli[sort_single(df.columns)])

pli abcdefghijklmno
[{810, 814}]


In [21]:
count_duplicates = 0
for i in pli[sort_single(df.columns)]:
    count_duplicates += (len (i) -1)
print (count_duplicates)

1
